## How to train your own model

Zacznijmyd od podstawowych importów i sprawdzenia jakie mamy możliwe środki do trenowania

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset
Dzisiaj będziemy pracować z bardziej skomplikowanym przykładem jakim jest [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html). Jest to zbiór obrazów o rozmiarze 32x32 pikseli, zawierający 10 klas obiektów. Możemy go pobrać z biblioteki torchvision.

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

In [ ]:
# teraz chcemy ustawić wielkość batcha i learning rate do trenowania

# TODO: spróbuj pozmieniać wartości tych parametrów
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# i przygotować dataset do załadowania

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=1
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=1
)

## Wizualizacja danych
Zobaczmy jak wyglądają nasze dane. W tym celu użyjemy biblioteki matplotlib, aby wyświetlić kilka obrazów wraz z podpisami z naszego zbioru danych.

In [ ]:
def show_images(images: torch.Tensor, labels: torch.Tensor, classes: list[str]) -> None:
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # permute to change from CxHxW to HxWxC
        plt.title(classes[labels[i]])
        plt.axis("off")
    plt.show()

In [ ]:
# zobaczmy pierwsze 10 obrazków z naszego datasetu

images, labels = next(iter(train_dataloader))
show_images(images[:8], labels[:8], train_dataloader.dataset.classes)

In [ ]:
class SimpleModel(torch.nn.Module):
    def __init__(self) -> None:
        super(SimpleModel, self).__init__()
        ### TODO: zdefiniuj prostą sieć neuronową z 2 ukrytymi warstwami o rozmiarach 128 i 64
        self.layer1 = torch.nn.Linear(32 * 32 * 3, 128)
        self.layer2 = torch.nn.Linear(128, 64)
        self.layer3 = torch.nn.Linear(64, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ### TODO: zaimplementuj funkcję forward z użyciem aktywacji ReLU pomiędzy warstwami, pamiętaj o spłaszczeniu wejścia
        x = x.flatten(start_dim=1)  # flatten: (batch, 3, 32, 32) -> (batch, 3072)
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)  # UWAGA: nie robimy tutaj softmax - CrossEntropyLoss wymaga wartości PRZED softmax
        return x

In [ ]:
def validation_loop(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
) -> tuple[float, float]:
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    return total_loss / len(dataloader), correct / total


def training_loop(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    num_epochs: int,
) -> None:
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        train_loss = total_loss / len(train_dataloader)
        val_loss, val_acc = validation_loop(model, test_dataloader, criterion)
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

In [ ]:
### TODO: trenowanie najmniejszego modelu
simple_model = SimpleModel().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(simple_model.parameters(), lr=learning_rate)

training_loop(simple_model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs)

In [ ]:
class SequentialModel(torch.nn.Module):
    def __init__(self, layers: list[int]) -> None:
        super(SequentialModel, self).__init__()

        ### TODO: zdefiniuj model sekwencyjny na podstawie podanej listy rozmiarów warstw, użyj aktywacji ReLU pomiędzy warstwami,
        ### hint: użyj torch.nn.Sequential oraz torch.nn.Linear
        all_layers = []
        input_size = 32 * 32 * 3

        for output_size in layers:
            all_layers.append(torch.nn.Linear(input_size, output_size))
            all_layers.append(torch.nn.ReLU())
            input_size = output_size

        all_layers.append(torch.nn.Linear(input_size, 10))
        self.model = torch.nn.Sequential(*all_layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ### TODO: zaimplementuj funkcję forward z użyciem spłaszczenia wejścia
        x = x.flatten(start_dim=1)  # flatten: (batch, 3, 32, 32) -> (batch, 3072)
        return self.model(x)

In [ ]:
model_bigger = SequentialModel([256, 128, 128, 128, 64]).to(device)
#### TODO: trenowanie większego modelu
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_bigger.parameters(), lr=learning_rate)

training_loop(model_bigger, train_dataloader, test_dataloader, criterion, optimizer, num_epochs)

## Strategia z ostatnich zajęć - konwolucje

In [ ]:
class ConvolutionModel(torch.nn.Module):
    def __init__(self) -> None:
        super(ConvolutionModel, self).__init__()

        ### TODO: zdefiniuj model konwolucyjny z 3 warstwami konwolucyjnymi
        ### Między konwolucjami użyj aktywacji ReLU oraz MaxPool2d z kernel_size=2
        ### Na koniec dodaj warstwę Linear do klasyfikacji na 10 klas
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = torch.nn.MaxPool2d(kernel_size=2)

        # Po 3 warstwach MaxPool2d: 32 -> 16 -> 8 -> 4
        self.fc = torch.nn.Linear(4 * 4 * 128, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ### TODO: zaimplementuj funkcję forward
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.flatten(start_dim=1)
        return self.fc(x)

In [ ]:
conv_model = ConvolutionModel().to(device)
### TODO: trenowanie modelu konwolucyjnego
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv_model.parameters(), lr=learning_rate)

training_loop(conv_model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs)